In [1]:
import quandl as ql
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime
import dask as dd
from dask import dataframe as ddf

%matplotlib inline
import seaborn as sns

start = datetime(2014, 1, 1)
end = datetime(2018, 12, 11)

key = 'YgkL6nxrm_SsUs7so2MM'


#Momentum Strategy
#1. Choose a universe of stocks, get daily prices
#2. Resample Daily prices and extract month-end-prices, compute log returns month-end resampled daily prices
#3. Rank by month-end returns selecting top n and bottom n
#4. Compute long and short portfolio returns (assume equal dollar amount of investment)
#5. Combine portfolio returns: Difference between short portfolio return and long portfolio return

#Test this at the start of every month but I really think this is a stupid idea. 
#What is the optimal period to select? Maybe there are more meaningful temporal definitions
#What is the optimal #number of assets to hold?
#Should we compare against some moving average estimate

In [2]:
def days_to_weeks(open_prices, high_prices, low_prices, close_prices):
    """Converts daily OHLC prices to weekly OHLC prices.
    
    Parameters
    ----------
    open_prices : DataFrame
        Daily open prices for each ticker and date
    high_prices : DataFrame
        Daily high prices for each ticker and date
    low_prices : DataFrame
        Daily low prices for each ticker and date
    close_prices : DataFrame
        Daily close prices for each ticker and date

    Returns
    -------
    open_prices_weekly : DataFrame
        Weekly open prices for each ticker and date
    high_prices_weekly : DataFrame
        Weekly high prices for each ticker and date
    low_prices_weekly : DataFrame
        Weekly low prices for each ticker and date
    close_prices_weekly : DataFrame
        Weekly close prices for each ticker and date
    """
    
    # TODO: Implement Function
    
    open_prices_weekly = open_prices.resample('10D').first()
    high_prices_weekly = high_prices.resample('10D').max()
    low_prices_weekly = low_prices.resample('10D').min()
    close_prices_weekly = close_prices.resample('10D').last()
    
    return open_prices_weekly, high_prices_weekly, low_prices_weekly, close_prices_weekly

In [3]:
company_list = pd.read_csv('/Users/justinsimcock/Downloads/companylist.csv')

In [2]:
#thank you to Kaggle for making this available. 
hist = pd.read_csv('/Users/justinsimcock/Downloads/daily-historical-stock-prices-1970-2018/historical_stock_prices.csv')

In [3]:
#set date to datetime object from a string
hist['date'] = pd.to_datetime(hist['date'])

In [4]:
#select the time period you want
#right now I am only selecting from 2017
post_2017_prices = hist.loc[hist.date >= '2017-01-01']
post_2017_prices = post_2017_prices.set_index('date')

In [5]:
len(post_2017_prices.ticker.unique())

5685

In [6]:
#Create our time series that we'll use for analysis. 
#We can test different time windows and this will be the next step
groupby_weekly_ticker = post_2017_prices.groupby('ticker').resample('W')
weekly_close_ticker = groupby_weekly_ticker.close.last().dropna(axis=0)

weekly_returns = compute_log_returns(weekly_close_ticker).unstack(level=0)
weekly_lookahead = compute_lookahead_returns(weekly_close_ticker, period=1).unstack(level=0)
top_weekly_n_long = get_top_n(weekly_returns, 50)
top_weekly_n_short = get_top_n(-weekly_returns, 50)
returns = portfolio_returns(top_weekly_n_long, top_weekly_n_short, weekly_lookahead, 100)

NameError: name 'compute_log_returns' is not defined

In [ ]:
#for xarray workflow...this is next
xr_weekly_close = weekly_close_ticker.to_xarray()

In [ ]:
top_weekly_n_short.iloc[0].sum()

In [ ]:
def compute_lookahead_returns(prices,period):
    '''
    Compute lookahead returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    period: int
        Periods to shift and compute lookahead
    Returns
    -------
    returns : DataFrame
        returns for each ticker and date
    '''
    
    return prices - prices.shift(period)

In [ ]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    prev_returns_copy = prev_returns.copy()
    cols = prev_returns_copy.columns
    for idx, r in prev_returns_copy.iterrows():
        n_largest = r.nlargest(top_n)

        prev_returns_copy.loc[idx, cols[cols.isin(n_largest.index)]] = np.int(1)
        prev_returns_copy.loc[idx, cols[~cols.isin(n_largest.index)]] = np.int(0)        
    
    return prev_returns_copy.astype(int)

In [ ]:
xr_log_returns = compute_log_returns_xr(xr_weekly_close, 1)

In [ ]:
# df_list = []
# for i in list_index:
#     symbol = company_list.iloc[i]['Symbol']
#     try:
#         series = web.DataReader(symbol, 'iex', start, end)
#     except Exception as e:
#         print(e)
#         continue

#     df = pd.DataFrame.from_dict({'symbol': [symbol], 'start': [series.iloc[0].close], 
#                           'end': [series.iloc[-1].close], 'change': [series.iloc[-1].close - series.iloc[0].close]})
                               
#     df_list.append(df)
                               

# strat_df = pd.concat(df_list, axis=0)

In [ ]:
def generate_positions(prices):
    """
    Generate the following signals:
     - Long 30 share of stock when the price is above 50 dollars
     - Short 10 shares when it's below 20 dollars
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    final_positions : DataFrame
        Final positions for each ticker and date
    """
    # TODO: Implement Functions
    
    pos_long = 30 * (prices > 50).astype(np.int)
    pos_short = -10 *(prices < 20).astype(np.int)
    
    return pos_long + pos_short

In [ ]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """    
    return (np.log(prices) - np.log(prices.shift(1)))

In [ ]:
def shift_returns(returns, shift_n):
    """
    Generate shifted returns
    
    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    shift_n : int
        Number of periods to move, can be positive or negative
    
    Returns
    -------
    shifted_returns : DataFrame
        Shifted returns for each ticker and date
    """    
    return returns.shift(shift_n)

In [ ]:
def compute_log_returns_xr(prices, shift_n):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : xr.DataArray
        Prices for each ticker and date
    
    shift_n: int
        intervals over which to shift
    
    Returns
    -------
    log_returns : xr.DataArray
        Log returns for each ticker and date
    """    
    return (np.log(prices) - np.log(prices.shift(date=shift_n)))

In [ ]:
def resample_prices(close_prices, freq='M'):
    """
    Resample close prices for each ticker at specified frequency.
    
    Parameters
    ----------
    close_prices : DataFrame
        Close prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    
    return close_prices.resample(freq).last()

In [ ]:
def portfolio_returns(df_long, df_short, lookahead_returns, n_stocks):
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """
    
    return (lookahead_returns*df_long)/n_stocks -  (lookahead_returns*df_short)/n_stocks

In [ ]:
from scipy import stats

def analyze_alpha(expected_portfolio_returns_by_date):
    """
    Perform a t-test with the null hypothesis being that the expected mean return is zero.
    
    Parameters
    ----------
    expected_portfolio_returns_by_date : Pandas Series
        Expected portfolio returns for each date
    
    Returns
    -------
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    t, p = stats.ttest_1samp(expected_portfolio_returns_by_date, 0.0)

    return t, p/2

In [ ]:
#Create our time series that we'll use for analysis. 
#We can test different time windows and this will be the next step
groupby_weekly_ticker = post_2017_prices.groupby('ticker').resample('W')
weekly_close_ticker = groupby_weekly_ticker.close.last().dropna(axis=0)

weekly_returns = compute_log_returns(weekly_close_ticker).unstack(level=0)
weekly_lookahead = compute_lookahead_returns(weekly_close_ticker, period=1).unstack(level=0)
top_weekly_n_long = get_top_n(weekly_returns, 50)
top_weekly_n_short = get_top_n(-weekly_returns, 50)
returns = portfolio_returns(top_weekly_n_long, top_weekly_n_short, weekly_lookahead, 100)